In [ ]:
!pip install requests beautifulSoup4 newspaper3k spacy googlesearch-python

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 27.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.6/97.6 kB 5.5 MB/s eta 0:00:00
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13540 sha256=f833011827a220cbecd2df1ef29e7fca072079c88cc0a3c3c8698bbf673fcedc
  Stored in directory: /root/.cache/pip/wheels/c8/d6/6c/384f58df48c00b9a31d638005143b5b3ac62c3d25fb1447f23
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-py3-none-any.whl size=3342 sha256=3980cce9de210033185300b9c1951ebee74b867115610bf2768d015e6fa034ae
  Stored in directory: /root/.cache/pip/wheels/97/02/e7/a1ff1760e12bdbaab0ac824fae5c1bc933e41

In [ ]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 34.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
!pip install sumy

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.3/97.3 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 51.3 MB/s eta 0:00:00
  Created wheel for breadability: filename=breadability-0.1.20-py2.py3-none-any.whl size=21691 sha256=ed0ec1142d43435fd01f6e94266f3c33bc93ad9f94797ebcb00dfa31f5c0b541
  Stored in directory: /root/.cache/pip/wheels/64/22/90/b84fcc30e16598db20a0d41340616dbf9b1e82bbcc627b0b33
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13704 sha256=0493a0a394587974f3ffcec63475070889c44fa0628bbede3435554501168250
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built breadability docopt


In [ ]:
import requests
from bs4 import BeautifulSoup
from googlesearch import search
import spacy
import newspaper
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict
import nltk
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.luhn import LuhnSummarizer
nltk.download('stopwords')
nltk.download('punkt')

nlp = spacy.load("en_core_web_sm")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# ***GOOGLE***

In [ ]:
def fetch_web(url):
  try:
    response=requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
    if(response.status_code!=200):
      return None
    return response.text
  except:
    return None

In [ ]:
def filter_url(urls):
  filtered_urls=[]
  for url in urls:
    html=fetch_web(url)
    if(html):
      soup=BeautifulSoup(html,'html.parser')
      if(soup.find_all('p')):
        filtered_urls.append(url)
  return filtered_urls

In [ ]:
def preprocess(text):
  tokens = nltk.word_tokenize(text.lower())
  stop_words = set(nltk.corpus.stopwords.words('english'))
  tokens = [token for token in tokens if token.isalnum() and token not in stop_words]
  return ' '.join(tokens)

In [ ]:
def analyze_similarity(content, query):
  query_preprocess=preprocess(query)
  content_preprocess=preprocess(content)
  doc=[query]+[content]
  vectorize=TfidfVectorizer().fit_transform(doc)
  vector=vectorize.toarray()
  query_vector=vector[0]
  content_vector=vector[1:]
  similarity=cosine_similarity([query_vector],content_vector).flatten()
  if(similarity>0.8):
    return content
  return ''

In [ ]:
import re
def extract_data(url):
  soup = BeautifulSoup(url, 'html.parser')
  for script in soup(["script", "style", "iframe", "header", "footer", "nav"]):
      script.extract()  # Removes these elements completely
  for element in soup.find_all(class_=["ad", "advertisement"]):
      element.extract()
  for element in soup.find_all(id=["ad", "advertisement"]):
      element.extract()
  # Remove references or citation patterns [number] or (number)
  text = re.sub(r'\[\d+\]|\(\d+\)', '', soup.get_text(separator=' '))

  # Clean up extra spaces and lines
  text = re.sub(r'\s+', ' ', text)
  return text.strip()

In [ ]:
def get_k_data(query,k):
  urls=list(search(query,num_results=int(k)))
  urlss=filter_url(urls)
  page_data=[]
  doc=nlp(query)
  keywords=[token.lemma_.lower() for token in doc if token.is_alpha and not token.is_stop]
  for url in urls:
    html=fetch_web(url)
    if(html):
      soup=BeautifulSoup(html,'html.parser')
      text=extract_data(html)
      page_data.append(text)
  return page_data

In [ ]:
def summary(all_data):
  parser = PlaintextParser.from_string(all_data, Tokenizer("english"))
  text=len(list(parser.document.sentences))
  sent_count=int(text*0.5)
  summarizer = LuhnSummarizer()
  summarys = summarizer(parser.document,sent_count)
  print(type(summarys))
  return ' '.join([str(sentence) for sentence in summarys])

In [ ]:
query=input("Enter the query:")
k=input("Enter the number of websites to search:")
url=get_k_data(query,k)
# print(url)
all_data=' '.join(urls for urls in url)
data=summary(all_data)

Enter the query:what is ai
Enter the number of websites to search:10
<class 'tuple'>


In [ ]:
data

'Artificial intelligence (AI) is a set of technologies that enable computers to perform a variety of advanced functions, including the ability to see , understand and translate spoken and written language , analyze data , make recommendations, and more. AI is the backbone of innovation in modern computing, unlocking value for individuals and businesses. For example, optical character recognition (OCR ) uses AI to extract text and data from images and documents, turns unstructured content into business-ready structured data, and unlocks valuable insights. New customers get $300 in free credits to spend on Google Cloud. Get started for free Stay informed 22:54 Introduction to generative AI Artificial intelligence defined Artificial intelligence is a field of science concerned with building computers and machines that can reason, learn, and act in such a way that would normally require human intelligence or that involves data whose scale exceeds what humans can analyze. AI is a broad fiel